In [1]:
from ift6758.data.acquisition import NHLGameData

In [2]:
data_path = './../ift6758/data/json_raw/'
data_fetcher = NHLGameData(data_path)

In [3]:
data_fetcher.fetch_season(2016);

Failed to retrieve data from https://statsapi.web.nhl.com/api/v1/game/2016021231/feed/live/.
Failed to fetch regular game 1231 for season 2016.
Failed to retrieve data from https://statsapi.web.nhl.com/api/v1/game/2016030147/feed/live/.
No data in playoff game 0147 for season 2016.
Failed to retrieve data from https://statsapi.web.nhl.com/api/v1/game/2016030176/feed/live/.
No data in playoff game 0176 for season 2016.
Failed to retrieve data from https://statsapi.web.nhl.com/api/v1/game/2016030251/feed/live/.
No data in playoff game 0251 for season 2016.
Failed to retrieve data from https://statsapi.web.nhl.com/api/v1/game/2016030261/feed/live/.
No data in playoff game 0261 for season 2016.
Failed to retrieve data from https://statsapi.web.nhl.com/api/v1/game/2016030271/feed/live/.
No data in playoff game 0271 for season 2016.
Failed to retrieve data from https://statsapi.web.nhl.com/api/v1/game/2016030281/feed/live/.
No data in playoff game 0281 for season 2016.
Failed to retrieve dat